In [65]:
import pandas as pd
import numpy as np
import toyplot
import time

In [2]:
%pwd

'/Users/patrickmckenzie/googledrive/projects/flower_color_phenology/notebooks'

In [3]:
%cd ..

/Users/patrickmckenzie/googledrive/projects/flower_color_phenology


# Open iNaturalist observations csv

In [4]:
inat_data = pd.read_csv('./data/combined_raw_inaturalist_export.csv')

/var/folders/qh/c7rwx84921n2kznn09zb2pnh0000gn/T/ipykernel_5464/2760840383.py:1: DtypeWarning: Columns (0,5,14,18,19,20,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  inat_data = pd.read_csv('./data/combined_raw_inaturalist_export.csv')


In [5]:
inat_data.columns

Index(['id', 'observed_on_string', 'observed_on', 'time_observed_at',
       'time_zone', 'user_id', 'user_login', 'user_name', 'created_at',
       'updated_at', 'quality_grade', 'license', 'url', 'image_url',
       'sound_url', 'tag_list', 'description', 'num_identification_agreements',
       'num_identification_disagreements', 'captive_cultivated',
       'oauth_application_id', 'place_guess', 'latitude', 'longitude',
       'positional_accuracy', 'private_place_guess', 'private_latitude',
       'private_longitude', 'public_positional_accuracy', 'geoprivacy',
       'taxon_geoprivacy', 'coordinates_obscured', 'positioning_method',
       'positioning_device', 'species_guess', 'scientific_name', 'common_name',
       'iconic_taxon_name', 'taxon_id'],
      dtype='object')

In [41]:
len(inat_data.index)

1763883

In [118]:
# let's (way later) make a column for the first two words of every scientific name in the data
inat_data['binomial'] = [' '.join(str(i).split( )[:2]) for i in inat_data.scientific_name]

# What are the most frequent species?

In [8]:
species, counts = np.unique([' '.join(str(i).split( )[:2]) for i in inat_data.scientific_name],return_counts=True)

In [11]:
species_num_obs_df = pd.DataFrame([species,counts],index=['species','num_obs']).T

In [16]:
species_num_obs_df.sort_values('num_obs',ascending=False)[:20]

,species,num_obs
13095,Trillium grandiflorum,12838
4142,Dipterostemon capitatus,12324
13092,Trillium erectum,10808
11480,Sanguinaria canadensis,10059
13102,Trillium ovatum,9984
5389,Ficaria verna,9840
3007,Claytonia virginica,9176
4873,Erodium cicutarium,8690
3729,Cypripedium acaule,8048
7097,Lamium purpureum,7742


Something interesting jumps out here. The most-observed species in our dataset of flowering-labeled North American iNat observatinos is Trillium grandiflorum. Overall in North America on iNat, there are 35,497 observations of this species. So **12,838 / 35,497 = 0.36 of these are labeled as flowering.**

In contrast, there 7,742 observations of Lamium purpureum labeled as flowering in this dataset, in contrast to 50,333 total observations of this species in North America that I see on the iNaturalist website. So **only 0.15 of the Lamium purpureum observations have phenology labels.**

Finally, let's look at white clover (Trifolium repens). Only 5,504 of those observations have flowering phenology labels, despite there being 79,678 observations. That amounts to only 0.07 of the observations having flowering phenology labels!

# What's the distribution of species abundance?

In [35]:
toyplot.bars(np.histogram(list(species_num_obs_df.num_obs),bins=100));

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t4369f8c9a66c4378961f096c0a536b18"> 0 4000 8000 12000 0 4000 8000 12000

# The VAST majority of species labeled as "flowering" are represented by very few records. Does this match iNaturalist observations of flowering plants overall?

# Another way to look at this:

In [36]:
species_num_obs_df

,species,num_obs
0,Abdra brachycarpa,13
1,Abelia ×,22
2,Abelmoschus,1
3,Abelmoschus esculentus,5
4,Abronia,2
...,...,...
13842,Zornia reticulata,2
13843,Zostera marina,2
13844,Zoysia matrella,1
13845,Zoysia pacifica,1


# 99,509 of the 1,763,883 observations come from the most common 10 species 

In [47]:
species_num_obs_df.sort_values('num_obs',ascending=False)[:10].num_obs.sum()

99509

# 298,820 of the 1,763,883 observations come from the most common 50 species 

In [44]:
species_num_obs_df.sort_values('num_obs',ascending=False)[:50].num_obs.sum()

298820

# 729,842 of the 1,763,883 observations come from the most common 250 species 

In [43]:
species_num_obs_df.sort_values('num_obs',ascending=False)[:250].num_obs.sum()

729842

# Remember: There are 13,846 total species in the dataset!

# Use pyinaturalist to get raw occurrence counts in inat

### First, subset our data:

In [70]:
subdf = species_num_obs_df.sample(1000)

In [71]:
import pyinaturalist

In [72]:
nelat = 54

nelng = -59

swlat = 24

swlng = -130


In [ ]:
trillium
counts_list = []
for species_idx in range(len(subdf.index)):
    taxon_name = subdf.species.iloc[species_idx]
    counts = pyinaturalist.get_observation_species_counts(quality_grade='research',
                                                          nelat=nelat,
                                                          nelng=nelng,
                                                          swlat=swlat,
                                                          swlng=swlng,
                                                          taxon_id=47125, # angiosperms
                                                          taxon_name=taxon_name
                                                         )
    counts_list.append(counts['total_results'])
    time.sleep(1)
    if not species_idx%50:
        print(species_idx)

In [81]:
taxon_name='Trillium grandiflorum'
counts = pyinaturalist.get_observation_species_counts(quality_grade='research',
                                                          nelat=nelat,
                                                          nelng=nelng,
                                                          swlat=swlat,
                                                          swlng=swlng,
                                                          taxon_id=47125, # angiosperms
                                                          taxon_name=taxon_name
                                                         )

In [85]:
counts['results'][0]['count']

34842

In [88]:
counts_list = []
for species_idx in range(len(subdf.index)):
    taxon_name = subdf.species.iloc[species_idx]
    counts = pyinaturalist.get_observation_species_counts(quality_grade='research',
                                                          nelat=nelat,
                                                          nelng=nelng,
                                                          swlat=swlat,
                                                          swlng=swlng,
                                                          taxon_id=47125, # angiosperms
                                                          taxon_name=taxon_name
                                                         )
    counts_list.append(counts['results'][0]['count'])
    time.sleep(1)
    if not species_idx%50:
        print(species_idx)

0
50
100
150
200
250
300
350
400
450
500


IndexError: list index out of range

In [90]:
species_idx

532

In [91]:
#counts_list = []
for species_idx in range(532,len(subdf.index)):
    taxon_name = subdf.species.iloc[species_idx]
    counts = pyinaturalist.get_observation_species_counts(quality_grade='research',
                                                          nelat=nelat,
                                                          nelng=nelng,
                                                          swlat=swlat,
                                                          swlng=swlng,
                                                          taxon_id=47125,  # angiosperms
                                                          taxon_name=taxon_name
                                                         )
    # Check if counts['results'] is not empty
    if counts['results']:
        counts_list.append(counts['results'][0]['count'])
    else:
        # Append nan if counts['results'] is empty
        counts_list.append(np.nan)

    time.sleep(1)
    if not species_idx % 50:
        print(species_idx)


550
600
650
700
750
800
850
900
950


In [96]:
toyplot.bars(np.histogram(np.array(counts_list)[~np.isnan(counts_list)],bins=20));

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ta2160a15f3a6473097aeacc4f2f30718"> 0 25000 50000 75000 0 300 600 900

Doesn't look that much different from the distribution of occurrences labeled as flowering.

# Is there a correlation between species that have many total observations and those that have many flowering-labeled observations?

In [98]:
subdf['total_counts_list'] = counts_list
subdf

,species,num_obs,total_counts_list
10307,Polygala verticillata,64,480.0
5039,Escobaria duncanii,1,33.0
9993,Physaria ovalifolia,6,115.0
2579,Ceanothus pauciflorus,120,709.0
6754,Ipomopsis congesta,37,713.0
...,...,...,...
2501,Castilleja salaisolaveae,11,11.0
9197,Paronychia albomarginata,4,4.0
7030,Koenigia phytolaccifolia,10,154.0
1963,Calochortus ownbeyi,4,4.0


In [102]:
toyplot.scatterplot(list(subdf.total_counts_list),list(subdf.num_obs),size=10);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tff15607590d14e6798e1c00bf671eaf1"> <g style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0" class="t

In [106]:
from scipy.stats import linregress

# Fitting a line
slope, intercept, r_value, p_value, std_err = linregress(list(subdf.dropna().total_counts_list),list(subdf.dropna().num_obs))

# Generate values for the best fit line
x_fit = np.linspace(subdf.dropna().total_counts_list.min(), subdf.dropna().total_counts_list.max(), 100)
y_fit = slope * x_fit + intercept

In [109]:
canvas = toyplot.Canvas(width=600, height=600)
axes = canvas.cartesian()
mark1 = axes.scatterplot(list(subdf.total_counts_list),list(subdf.num_obs),size=10)
mark2 = axes.plot(x_fit,y_fit)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="600.0px" viewBox="0 0 600.0 600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td607f3c1b6034194889485053b9d3a64"> <g style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0"

# Now let's introduce color to the equation...

In [112]:
color_matching = pd.read_csv('./data/cleaned_matched_colors.csv',index_col=0)

In [120]:
num_occs = []
for row in list(color_matching.species_name):
    num_occs.append(len(inat_data[inat_data.binomial.eq(row.capitalize())].index))

In [123]:
color_matching['num_occs_flowering'] = num_occs

In [125]:
red_df = color_matching[color_matching.color.eq('red')]

In [126]:
red_df

,species_name,color,num_occs_flowering
0,abelmoschus esculentus,red,5
34,achillea millefolium,red,5680
40,achyranthes aspera,red,2
49,adonis aestivalis,red,1
52,adonis aestivalis,red,1
...,...,...,...
3184,trifolium pratense,red,6053
3203,tropaeolum majus,red,486
3242,vaccinium angustifolium,red,373
3361,watsonia meriana,red,1


In [127]:
red_counts_list = []
for species_idx in range(len(red_df.index)):
    taxon_name = red_df.species_name.iloc[species_idx].capitalize()
    counts = pyinaturalist.get_observation_species_counts(quality_grade='research',
                                                          nelat=nelat,
                                                          nelng=nelng,
                                                          swlat=swlat,
                                                          swlng=swlng,
                                                          taxon_id=47125,  # angiosperms
                                                          taxon_name=taxon_name
                                                         )
    # Check if counts['results'] is not empty
    if counts['results']:
        red_counts_list.append(counts['results'][0]['count'])
    else:
        # Append nan if counts['results'] is empty
        red_counts_list.append(np.nan)

    time.sleep(1)
    if not species_idx % 50:
        print(species_idx)


0
50
100


IndexError: single positional indexer is out-of-bounds

In [130]:
red_df['num_occs_total'] = red_counts_list

/var/folders/qh/c7rwx84921n2kznn09zb2pnh0000gn/T/ipykernel_5464/4112548169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  red_df['num_occs_total'] = red_counts_list


In [131]:
red_df

,species_name,color,num_occs_flowering,num_occs_total
0,abelmoschus esculentus,red,5,190
34,achillea millefolium,red,5680,90092
40,achyranthes aspera,red,2,59
49,adonis aestivalis,red,1,6
52,adonis aestivalis,red,1,6
...,...,...,...,...
3184,trifolium pratense,red,6053,61194
3203,tropaeolum majus,red,486,9536
3242,vaccinium angustifolium,red,373,6554
3361,watsonia meriana,red,1,29


In [132]:
canvas = toyplot.Canvas(width=600, height=600)
axes = canvas.cartesian()
mark1 = axes.scatterplot(list(red_df.num_occs_total),list(red_df.num_occs_flowering),size=10)
#mark2 = axes.plot(x_fit,y_fit)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="600.0px" viewBox="0 0 600.0 600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t96a863cb384545549c931a04b2703031"> 0 30000 60000 90000 0 2000 4000 6000

In [162]:
from scipy.stats import linregress

# Fitting a line
slope, intercept, r_value, p_value, std_err = linregress(list(red_df.num_occs_total),list(red_df.num_occs_flowering))

# Generate values for the best fit line
red_x_fit = np.linspace(red_df.num_occs_total.min(), red_df.num_occs_total.max(), 100)
red_y_fit = slope * red_x_fit + intercept

In [163]:
std_err

0.0025243055287638098

In [138]:
white_df = color_matching[color_matching.color.eq('white')].drop_duplicates()
len(white_df.index)

464

In [139]:
white_counts_list = []
for species_idx in range(len(white_df.index)):
    taxon_name = white_df.species_name.iloc[species_idx].capitalize()
    counts = pyinaturalist.get_observation_species_counts(quality_grade='research',
                                                          nelat=nelat,
                                                          nelng=nelng,
                                                          swlat=swlat,
                                                          swlng=swlng,
                                                          taxon_id=47125,  # angiosperms
                                                          taxon_name=taxon_name
                                                         )
    # Check if counts['results'] is not empty
    if counts['results']:
        white_counts_list.append(counts['results'][0]['count'])
    else:
        # Append nan if counts['results'] is empty
        white_counts_list.append(np.nan)

    time.sleep(1)
    if not species_idx % 50:
        print(species_idx)


0
50
100
150
200
250
300
350
400
450


In [140]:
white_df['num_occs_total'] = white_counts_list

In [154]:
white_df = white_df.dropna()

In [155]:
from scipy.stats import linregress

# Fitting a line
slope, intercept, r_value, p_value, std_err = linregress(list(white_df.num_occs_total),list(white_df.num_occs_flowering),)

# Generate values for the best fit line
white_x_fit = np.linspace(white_df.num_occs_total.min(), white_df.num_occs_total.max(), 100)
white_y_fit = slope * white_x_fit + intercept

In [161]:
std_err

0.002034308211526399

In [166]:
# show the relationship 
canvas = toyplot.Canvas(width=600, height=600)
axes = canvas.cartesian(xlabel = 'num_total_occurrences',ylabel='num_labeled_flowering')
mark3 = axes.scatterplot(list(white_df.num_occs_total),list(white_df.num_occs_flowering),size=10,color='black')
mark4 = axes.plot(white_x_fit,white_y_fit,color='black')
mark1 = axes.scatterplot(list(red_df.num_occs_total),list(red_df.num_occs_flowering),size=10,color='red')
mark2 = axes.plot(red_x_fit,red_y_fit,color='red')

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="600.0px" viewBox="0 0 600.0 600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t468b9361d39b45c793f0b94afbf90b57"> <path d="M 50.00554988234249 549.1360208599576 L 55.05599887342994 544.8701928626786 L 60.10644786451739 540.6043648653992 L 65.15689685560484 536.33853686812 L 70.20734584669229 532.0727088708409 L 75.25779483777974 527.8068808735617 L 80.30824382886719 523.5410528762824 L 85.35869281995465 519.2752248790032 L 90.40914181104208 515.009396881724 L 95.45959080212954 510.7435688844448 L 100.51003979321699 506.47774088716557 L 105.56048878430444 502.2119128898864 L 110.61093777539187 497.9460848926072 L 115.66138676647934 493.680256895328 L 120.71183575756679 489.41442889804875 L 125.76228474865422 485.14860090076957 L 130.8127337397417 480.8827729034904 L 135.86318273082915 476.61694490621113 L 140.9136317219166 472.351116908932 L 145.96408071300402 468.08528891165275 L 151.01452970409147 463.8194609143735 L 156.06497869517892 459.5536329170944 L 161.1154276862664 455.28780491981513 L 166.16587667735385 451.02197692253594 L 171.21632566844127 446.75614892525664 L 176.26677465952875 442.4903209279775 L 181.31722365061617 438.22449293069826 L 186.36767264170362 433.9586649334191 L 191.4181216327911 429.6928369361399 L 196.46857062387852 425.42700893886064 L 201.519019614966 421.16118094158145 L 206.56946860605342 416.89535294430226 L 211.6199175971409 412.62952494702307 L 216.67036658822835 408.36369694974377 L 221.7208155793158 404.0978689524647 L 226.77126457040325 399.8320409551854 L 231.82171356149067 395.56621295790626 L 236.87216255257812 391.30038496062696 L 241.9226115436656 387.0345569633478 L 246.97306053475305 382.7687289660686 L 252.0235095258405 378.50290096878933 L 257.0739585169279 374.2370729715102 L 262.12440750801534 369.971244974231 L 267.1748564991029 365.70541697695177 L 272.2253054901903 361.4395889796726 L 277.2757544812777 357.17376098239333 L 282.3262034723652 352.90793298511414 L 287.3766524634526 348.64210498783495 L 292.4271014545401 344.37627699055577 L 297.4775504456275 340.1104489932765 L 302.527999436715 335.8446209959974 L 307.5784484278024 331.57879299871814 L 312.6288974188899 327.3129650014389 L 317.6793464099773 323.0471370041597 L 322.7297954010648 318.78130900688046 L 327.7802443921522 314.51548100960133 L 332.8306933832397 310.249653012322 L 337.8811423743271 305.98382501504295 L 342.93159136541453 301.71799701776365 L 347.98204035650207 297.4521690204844 L 353.0324893475895 293.1863410232053 L 358.0829383386769 288.920513025926 L 363.1333873297644 284.65468502864684 L 368.18383632085187 280.38885703136765 L 373.23428531193923 276.12302903408846 L 378.28473430302677 271.8572010368092 L 383.3351832941142 267.59137303953 L 388.3856322852016 263.32554504225084 L 393.4360812762891 259.05971704497153 L 398.4865302673765 254.7938890476924 L 403.536979258464 250.52806105041313 L 408.5874282495514 246.26223305313394 L 413.63787724063883 241.9964050558548 L 418.6883262317263 237.7305770585756 L 423.7387752228138 233.46474906129635 L 428.78922421390126 229.19892106401716 L 433.8396732049886 224.9330930667379 L 438.8901221960761 220.66726506945867 L 443.9405711871636 216.40143707217953 L 448.991020178251 212.13560907490032 L 454.0414691693385 207.86978107762113 L 459.09191816042596 203.60395308034185 L 464.1423671515134 199.33812508306272 L 469.19281614260086 195.07229708578353 L 474.24326513368817 190.8064690885043 L 479.2937141247757 186.54064109122504 L 484.3441631158632 182.2748130939459 L 489.39461210695055 178.0089850966667 L 494.4450610980381 173.743157099387